# Projeto TCC: Reconhecimento Automático de Libras com IA

## Fase 1: Configuração e Análise dos Dados

**Objetivo:** Carregar e explorar o dataset Libras MNIST para entender a estrutura dos dados.

**Dataset:** Libras MNIST do Kaggle - https://www.kaggle.com/datasets/datamoon/libras-mnist


## 1. Importação das Bibliotecas Necessárias


In [ ]:
# Bibliotecas fundamentais
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import os
import zipfile
from pathlib import Path

# TensorFlow e Keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical

# Scikit-learn para métricas
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

# Configurações para melhor visualização
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

# Verificar versões
print(f"TensorFlow: {tf.__version__}")
print(f"OpenCV: {cv2.__version__}")
print(f"NumPy: {np.__version__}")
print(f"Pandas: {pd.__version__}")

# Verificar se GPU está disponível
print(f"\nGPU disponível: {tf.config.list_physical_devices('GPU')}")
if tf.config.list_physical_devices('GPU'):
    print("✅ GPU detectada! O treinamento será mais rápido.")
else:
    print("⚠️  Nenhuma GPU detectada. O treinamento pode ser mais lento.")


## 2. Configuração do Kaggle API

Para baixar o dataset, você precisará:
1. Criar uma conta no Kaggle
2. Baixar seu arquivo de credenciais (kaggle.json)
3. Fazer upload do arquivo para o Colab


## 3. Download do Dataset Libras MNIST


In [ ]:
# Download do dataset
!kaggle datasets download -d datamoon/libras-mnist

# Extrair o arquivo zip
with zipfile.ZipFile('libras-mnist.zip', 'r') as zip_ref:
    zip_ref.extractall('.')

# Verificar o conteúdo extraído
print("📂 Conteúdo do dataset:")
!ls -la

# Listar arquivos CSV
csv_files = [f for f in os.listdir('.') if f.endswith('.csv')]
print(f"\n📊 Arquivos CSV encontrados: {csv_files}")


## 4. Carregamento e Exploração dos Dados


In [ ]:
# Carregar os dados
try:
    # Tentar carregar o arquivo principal
    df = pd.read_csv('libras_mnist.csv')
    print("✅ Dataset carregado com sucesso!")
except FileNotFoundError:
    # Se não encontrar, listar todos os CSVs disponíveis
    csv_files = [f for f in os.listdir('.') if f.endswith('.csv')]
    print(f"❌ Arquivo 'libras_mnist.csv' não encontrado.")
    print(f"📋 Arquivos CSV disponíveis: {csv_files}")
    if csv_files:
        df = pd.read_csv(csv_files[0])
        print(f"✅ Carregando {csv_files[0]} como alternativa.")

# Informações básicas do dataset
print(f"\n📊 Informações do Dataset:")
print(f"Dimensões: {df.shape}")
print(f"Colunas: {list(df.columns)}")
print(f"\nPrimeiras 5 linhas:")
print(df.head())

# Verificar se há valores nulos
print(f"\n🔍 Valores nulos por coluna:")
print(df.isnull().sum())

# Informações sobre os tipos de dados
print(f"\n📋 Tipos de dados:")
print(df.dtypes)


## 5. Análise das Classes (Letras de Libras)


In [ ]:
# Identificar a coluna de labels (geralmente a primeira ou última)
label_column = df.columns[0]  # Assumindo que a primeira coluna é o label
print(f"🏷️  Coluna de labels identificada: '{label_column}'")

# Análise das classes
unique_labels = df[label_column].unique()
n_classes = len(unique_labels)

print(f"\n📚 Classes encontradas: {n_classes}")
print(f"Labels únicos: {sorted(unique_labels)}")

# Distribuição das classes
class_counts = df[label_column].value_counts().sort_index()
print(f"\n📊 Distribuição das classes:")
print(class_counts)

# Visualização da distribuição
plt.figure(figsize=(15, 6))
plt.subplot(1, 2, 1)
class_counts.plot(kind='bar', color='skyblue', edgecolor='black')
plt.title('Distribuição das Classes de Libras')
plt.xlabel('Classe (Letra)')
plt.ylabel('Quantidade de Amostras')
plt.xticks(rotation=45)
plt.grid(axis='y', alpha=0.3)

plt.subplot(1, 2, 2)
plt.pie(class_counts.values, labels=class_counts.index, autopct='%1.1f%%', startangle=90)
plt.title('Proporção das Classes')

plt.tight_layout()
plt.show()

# Estatísticas da distribuição
print(f"\n📈 Estatísticas da distribuição:")
print(f"Média de amostras por classe: {class_counts.mean():.1f}")
print(f"Desvio padrão: {class_counts.std():.1f}")
print(f"Mínimo: {class_counts.min()}")
print(f"Máximo: {class_counts.max()}")
print(f"Classes com menos de 100 amostras: {(class_counts < 100).sum()}")


## 6. Visualização de Amostras das Imagens


In [ ]:
# Preparar os dados para visualização
# Assumindo que as colunas de pixel começam após a coluna de label
pixel_columns = df.columns[1:]  # Todas as colunas exceto a primeira (label)
print(f"🖼️  Colunas de pixel: {len(pixel_columns)} (de {pixel_columns[0]} a {pixel_columns[-1]})")

# Determinar dimensões da imagem baseado no número de pixels
n_pixels = len(pixel_columns)
img_size = int(np.sqrt(n_pixels))
print(f"📐 Dimensões da imagem: {img_size}x{img_size} pixels")

# Função para reconstruir imagem a partir dos pixels
def reconstruct_image(pixel_row, img_size):
    """Reconstrói uma imagem a partir de uma linha de pixels"""
    return pixel_row.values.reshape(img_size, img_size)

# Visualizar amostras de cada classe
fig, axes = plt.subplots(4, 8, figsize=(20, 10))
axes = axes.ravel()

# Pegar uma amostra de cada classe
for i, label in enumerate(sorted(unique_labels)):
    if i >= 32:  # Limitar a 32 classes para visualização
        break
    
    # Pegar a primeira amostra desta classe
    sample = df[df[label_column] == label].iloc[0]
    
    # Reconstruir a imagem
    image = reconstruct_image(sample[pixel_columns], img_size)
    
    # Plotar
    axes[i].imshow(image, cmap='gray')
    axes[i].set_title(f'Classe: {label}', fontsize=10)
    axes[i].axis('off')

# Ocultar eixos extras se necessário
for i in range(len(unique_labels), 32):
    axes[i].axis('off')

plt.suptitle('Amostras do Dataset Libras MNIST', fontsize=16, y=0.98)
plt.tight_layout()
plt.show()

# Visualizar múltiplas amostras da mesma classe
print("\n🔄 Múltiplas amostras da mesma classe (exemplo):")
example_class = sorted(unique_labels)[0]  # Primeira classe
class_samples = df[df[label_column] == example_class].head(8)

fig, axes = plt.subplots(2, 4, figsize=(12, 6))
axes = axes.ravel()

for i, (_, sample) in enumerate(class_samples.iterrows()):
    image = reconstruct_image(sample[pixel_columns], img_size)
    axes[i].imshow(image, cmap='gray')
    axes[i].set_title(f'Amostra {i+1}', fontsize=10)
    axes[i].axis('off')

plt.suptitle(f'Múltiplas amostras da classe {example_class}', fontsize=14)
plt.tight_layout()
plt.show()


## 7. Preparação dos Dados para o Modelo


In [ ]:
# Separar features (pixels) e labels
X = df[pixel_columns].values
y = df[label_column].values

print(f"📊 Dados preparados:")
print(f"Features (X): {X.shape}")
print(f"Labels (y): {y.shape}")
print(f"Tipo de dados X: {X.dtype}")
print(f"Tipo de dados y: {y.dtype}")

# Verificar range dos valores de pixel
print(f"\n🔍 Análise dos valores de pixel:")
print(f"Valor mínimo: {X.min()}")
print(f"Valor máximo: {X.max()}")
print(f"Valor médio: {X.mean():.2f}")
print(f"Desvio padrão: {X.std():.2f}")

# Reshape para formato de imagem
X_images = X.reshape(-1, img_size, img_size)
print(f"\n🖼️  Imagens reshapeadas: {X_images.shape}")

# Normalizar os pixels para o range [0, 1]
X_normalized = X_images.astype('float32') / 255.0
print(f"✅ Pixels normalizados para o range [0, 1]")
print(f"Novo range: [{X_normalized.min():.3f}, {X_normalized.max():.3f}]")

# Converter labels para categorical (one-hot encoding)
y_categorical = to_categorical(y, num_classes=n_classes)
print(f"\n🏷️  Labels convertidos para categorical: {y_categorical.shape}")
print(f"Exemplo de label original: {y[0]} -> one-hot: {y_categorical[0]}")

# Dividir em treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    X_normalized, y_categorical, 
    test_size=0.2, 
    random_state=42, 
    stratify=y  # Manter proporção das classes
)

print(f"\n📚 Divisão dos dados:")
print(f"Treino: {X_train.shape[0]} amostras")
print(f"Teste: {X_test.shape[0]} amostras")
print(f"Proporção treino/teste: {X_train.shape[0]/X_test.shape[0]:.1f}:1")

# Verificar se a divisão mantém a proporção das classes
train_labels = np.argmax(y_train, axis=1)
test_labels = np.argmax(y_test, axis=1)

print(f"\n✅ Verificação da estratificação:")
print(f"Classes no treino: {len(np.unique(train_labels))}")
print(f"Classes no teste: {len(np.unique(test_labels))}")


## 8. Resumo da Fase 1

### ✅ O que foi realizado:
1. **Configuração do ambiente** com todas as bibliotecas necessárias
2. **Download e carregamento** do dataset Libras MNIST
3. **Análise exploratória** dos dados:
   - Estrutura do dataset
   - Distribuição das classes
   - Visualização de amostras
4. **Preparação dos dados** para o modelo:
   - Normalização dos pixels
   - Conversão para formato de imagem
   - One-hot encoding dos labels
   - Divisão treino/teste

### 📊 Características do Dataset:
- **Classes:** 24 letras do alfabeto de Libras
- **Dimensões das imagens:** 28x28 pixels (formato MNIST)
- **Formato:** Escala de cinza (1 canal)
- **Distribuição:** Relativamente balanceada entre as classes

### 🚀 Próximos Passos (Fase 2):
1. Adaptar as imagens para o formato RGB (3 canais) para o MobileNetV2
2. Redimensionar para 224x224 pixels
3. Implementar o modelo com Transfer Learning
4. Treinar e avaliar o modelo

---

**💡 Dica:** Salve este notebook e prossiga para a Fase 2 quando estiver pronto!


In [ ]:
# Instalar e configurar Kaggle API
!pip install kaggle

# Criar diretório .kaggle
!mkdir -p ~/.kaggle

# INSTRUÇÕES PARA O USUÁRIO:
# 1. Faça upload do arquivo kaggle.json para o Colab
# 2. Execute a célula abaixo após o upload

# Descomente e execute após fazer upload do kaggle.json:
# !cp kaggle.json ~/.kaggle/
# !chmod 600 ~/.kaggle/kaggle.json

print("📁 Configure o Kaggle API seguindo as instruções acima.")
print("📋 Depois, descomente as linhas de configuração e execute novamente.")
